## Collecting Anomaly Patterns for Evaludation of AD

In [30]:
%reload_ext autoreload
%autoreload 2

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

In [32]:
import sys
sys.path.append('../')
from tsdr import tsdr
from eval import groundtruth

In [33]:
import pandas as pd
from collections import defaultdict

In [34]:
from meltria import loader

metrics_files = !find /datasets/argowf-chaos-rq54b/ -type f -name "*.json"
dataset_generator = loader.load_dataset_as_generator(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=120,
)
rerords = [r for rec in dataset_generator for r in rec]

In [35]:
from meltria.loader import DatasetRecord

route_by_record: dict[DatasetRecord, list[tuple[list[str], pd.DataFrame]]] = defaultdict(list)

for record in rerords:
    filtered_df: pd.DataFrame = tsdr.filter_out_no_change_metrics(record.data_df, parallel=True)
    gt_candidates = groundtruth.select_ground_truth_metrics_in_routes(record.pk, filtered_df.columns.to_list(), record.chaos_type(), record.chaos_comp())
    for routes, _ in gt_candidates:
        route_by_record[record].append((routes, filtered_df))

In [36]:
import ipywidgets as widgets
from IPython.display import display

In [37]:
SAVE_DIR = "../samples/anomaly_patterns"

In [38]:
import datetime
import jsonlines
import glob

In [39]:
def create_widget(yield_on_click) -> widgets.Box:
    save_button = widgets.Button(description='Save')
    skip_button = widgets.Button(description='Skip')
    select_pattern = widgets.Select(
        options=[
            'Sudden increase', 'Sudden decrease', 'Level shift up', 'Level shift down', 
            'Steady increase', 'Steady decrease', 'Single spike', 'Single dip',
            'Transient level shift up', 'Transient level shift down', 'Multiple spikes', 'Multiple dips', 'Fluctuations',
            'White noise', 'Other normal',
        ],
        rows=15,
        layout=widgets.Layout(width='20%'),
    )
    select_position = widgets.Select(
        options=["no_anomaly", "anomaly_during_fault", "anomaly_outside_fault"],
        layout=widgets.Layout(width='15%'),
    )
    output = widgets.Output(layout={'border': '1px solid black'})

    fig = plt.figure(figsize=(6, 8), clear=True)
    ax = fig.add_subplot(2, 1, 1)
    ax_sli = fig.add_subplot(2, 1, 2)

    def show() -> None:
        target_app, chaos_type, chaos_comp, metric, ts, sli_ts = next(yield_on_click)
        ax.clear()
        ax.set_title(f"{chaos_type}/{chaos_comp}\n{metric}\n")
        ax.plot(ts)
        ax_sli.clear()
        ax_sli.plot(sli_ts)
        ax_sli.set_title(f"SLI: {chaos_type}/{chaos_comp}")
        for _ax in [ax, ax_sli]:
            _ax.axvspan(100, len(ts), color='red', alpha=0.5)
        with output:
            display(fig)

    def on_save_click_callback(clicked_button: widgets.Button) -> None:
        yield_on_click.send((select_position.value, select_pattern.value))
        with output:
            output.clear_output(wait=True)
            print(f"Selected {select_pattern.value} and {select_position.value}!")
        show()

    save_button.on_click(on_save_click_callback)
    plt.close(fig=fig)
    show()

    def on_skip_click_callback(clicked_button: widgets.Button) -> None:
        with output:
            output.clear_output(wait=True)
            print(f"Skipped")
        show()

    skip_button.on_click(on_skip_click_callback)

    return widgets.HBox([select_position, select_pattern, widgets.VBox([save_button, skip_button]), output])

In [40]:
def gen_time_series(routes_by_record):
    now = datetime.datetime.today().strftime('%Y%m%d-%H%M%S')
    save_file_name = f"{SAVE_DIR}/anomaly_patterns_{now}.jsonl"
    # append mode
    writer = jsonlines.open(save_file_name, mode='a', flush=True)

    for record, routes in routes_by_record.items():
        unique_metric_check = {}
        sli_metric: str = "m-ts-ui-dashboard_nginx_http_response_count_total"
        sli_ts = record.data_df.loc[:, sli_metric].to_numpy()
        for metrics, filtered_df in routes:
            # sli_metric: str = record.pk.get_root_metrics()[0]
            for metric in metrics:
                # skip duplicated metric
                if metric in unique_metric_check and unique_metric_check[metric]:
                    continue
                unique_metric_check[metric] = True

                ts = filtered_df.loc[:, metric].to_numpy()
                v = (yield (record.target_app(), record.chaos_type(), record.chaos_comp(), metric, ts, sli_ts))
                if v is None:
                    continue
                position, pattern_name = v
                writer.write({
                    'target_app': record.target_app(), 
                    'chaos_type': record.chaos_type(),
                    'chaos_comp': record.chaos_comp(), 
                    'metric': metric,
                    'anomaly_position': position,
                    'anomaly_pattern': pattern_name,
                    'time_series': ts.tolist(),
                })
    writer.close()

In [41]:
box = create_widget(gen_time_series(route_by_record))
display(box)